# Environment Setup

---


In [2]:
pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [50]:
!pip install imblearn

In [1]:
!pip install wandb -qU

In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"mammomind10","key":"dd6c18f89704c6ce79e620ca1a78991c"}'}

In [4]:
#Install Keggle library
! pip install kaggle

In [5]:
#create directory name keggle
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
#Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets list

ref                                                        title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                             Apple Quality                                     170KB  2024-01-11 14:31:07          11658        273  1.0              
kanchana1990/2024-amazon-best-sellers-top-valentine-gifts  2024 Amazon Best Sellers: Top Valentine Gifts 🎯💘   25KB  2024-02-06 18:49:26            523         31  1.0              
asaniczka/mountain-climbing-accidents-dataset              Mountain Climbing Accidents Dataset                19KB  2024-02-07 23:54:04            344         25  1.0              
hummaamqaasim/jobs-in-data                                 Jobs and Salaries in Data Science   

In [9]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

creditcardfraud.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
import zipfile
zip_ref = zipfile.ZipFile('creditcardfraud.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Imports

---


In [11]:
import wandb
import numpy as np
import torch
import torch.nn as nn

import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader

# WandB - Lognig and Initialization

---


In [12]:
wandb.login()

wandb: Currently logged in as: mammomind1-0 (mammomind). Use `wandb login --relogin` to force relogin


True

In [63]:
wandb.init(
    project = 'CreditCard_Fraud_Detection',
    config={
        "epochs": 50,
        "batch_size": 64,
        "lr": 0.0001
    }
)
config = wandb.config

# Data Upload and DataFrame Creation

In [14]:
# Importing Data

data = pd.read_csv('/content/creditcard.csv')

In [15]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Preprocessing

---


In [16]:
std_scaler = StandardScaler()
rob_scaler = RobustScaler() # RobustScaler is less prone to outliers.


data['scaled_amount'] = rob_scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['scaled_time'] = rob_scaler.fit_transform(data['Time'].values.reshape(-1,1))

data.drop(['Time','Amount'], axis=1, inplace=True)

In [17]:
scaled_amount = data['scaled_amount']
scaled_time = data['scaled_time']

data.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
data.insert(0, 'scaled_amount', scaled_amount)
data.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

data.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [18]:
# Seperating predictor and traget variables
print(type(data))
X = data.iloc[:,0:30]
Y = list(data.iloc[:,-1])

<class 'pandas.core.frame.DataFrame'>


In [19]:
X_num = X.to_numpy()

In [20]:
X_num.shape

(284807, 30)

In [21]:
type(X_num)

numpy.ndarray

In [22]:
# Converting Y variable into a numpy array
Y_num = np.array(Y)
# type(X)

In [23]:
type(Y_num)

numpy.ndarray

In [24]:
test_size = 0.2  # 20% for test set
# val_size = 0.2  # 10% for validation set
train_size = 1 - test_size # - val_size

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_num, Y_num, test_size=test_size, random_state=42)  # Set random seed for reproducibility
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size / train_size, random_state=42)

In [26]:
import imblearn
print(imblearn.__version__)

0.12.0


In [30]:
from imblearn.over_sampling import SMOTE

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


Before OverSampling, counts of label '1': 394
Before OverSampling, counts of label '0': 227451 

After OverSampling, the shape of train_X: (454902, 30)
After OverSampling, the shape of train_y: (454902,) 

After OverSampling, counts of label '1': 227451
After OverSampling, counts of label '0': 227451


In [31]:
# convert everything into tensors

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

X_train = torch.tensor(X_train_res, device=device)  # Set appropriate data type and device
X_test = torch.tensor(X_test,  device=device)
y_train = torch.tensor(y_train_res,  device=device)  # Assume numerical labels (adjust if non-numerical)
y_test = torch.tensor(y_test,  device=device)

if "Y_val" in locals():  # Handle validation set if it exists
    X_val = torch.tensor(X_val, device=device)
    y_val = torch.tensor(y_val, device=device)


In [32]:
print(y_train)

tensor([0, 0, 0,  ..., 1, 1, 1])


In [33]:
print(X_train)

tensor([[-0.1678,  0.6892,  1.9550,  ..., -0.2168,  0.0452, -0.0471],
        [ 0.3340,  0.3816, -0.4010,  ...,  0.3619, -0.3705, -0.1448],
        [-0.1399,  0.7621,  0.0725,  ...,  0.1197,  0.2064,  0.0703],
        ...,
        [-0.2864,  0.7176, -1.6575,  ...,  0.4975,  0.5424,  0.0276],
        [-0.0473, -0.5770,  0.3321,  ...,  0.2251,  0.5516,  0.3050],
        [-0.2538,  0.4361,  1.8925,  ...,  0.2702,  0.0278,  0.0309]],
       dtype=torch.float64)


In [34]:
print(y_train.shape)
print(y_test.shape)
# print(y_val.shape)

torch.Size([454902])
torch.Size([56962])


In [35]:
# transforming y valriable into a 2D tensor

y_train = y_train.unsqueeze(1)
y_test = y_test.unsqueeze(1)

In [36]:
print(y_train.shape)
print(y_test.shape)
# print(y_val.shape)

torch.Size([454902, 1])
torch.Size([56962, 1])


In [37]:
print(X_train.shape)
print(X_test.shape)

torch.Size([454902, 30])
torch.Size([56962, 30])


# Dataset Creation
---


In [38]:
# create and load mini batches

class CrediCardDataset(Dataset):
  def __init__(self, x, y ):
    self.x = x
    self.y = y

  # return x and y for given index
  def __getitem__(self, index):
    return self.x[index], self.y[index]


  def __len__(self):
    return len(self.x)

In [39]:
# create an actual dataet  from the class

train_dataset = CrediCardDataset(X_train, y_train)
test_dataset = CrediCardDataset(X_test, y_test)

In [40]:
train_dataset.__getitem__(5)

(tensor([ 0.2546, -0.5471, -0.7951,  1.0618,  0.6408,  0.5946,  0.5516,  0.0976,
          0.6335,  0.0838, -0.0275,  0.3963, -1.2064, -0.9377, -1.0377,  0.4163,
          1.5069, -0.3381, -0.1704,  0.0439,  0.3474,  0.0931,  0.0265,  0.3212,
         -0.2470, -0.7799, -0.1602, -0.2365,  0.4427,  0.2681],
        dtype=torch.float64),
 tensor([0]))

In [64]:
# Iterable data components for training ( mini batches )

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

In [65]:
# demonstrate the data iterating

for (x_batch,y_batch) in train_loader:
  print(f"X shape: {x_batch.shape}")
  print(f"y shape: {y_batch.shape}")
  break

X shape: torch.Size([64, 30])
y shape: torch.Size([64, 1])


# Define Neural Network Architecture
---


In [66]:
# neural network class

class Perceptron(nn.Module):

  def __init__(self,input_features,dropout_prob=0.4):
    # inherit all the features from parent class
    super(Perceptron,self).__init__()
    self.fc1 = nn.Linear(input_features, 5)
    self.fc2 = nn.Linear(5, 4)
    self.fc3 = nn.Linear(4, 3)
    self.fc4 = nn.Linear(3, 1)
    # self.tanh = nn.Tanh()
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.dropout = nn.Dropout(p=dropout_prob)

  # method to build the network flow
  def forward (self, x ):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.dropout(out)

    out = self.fc2(out)
    out = self.relu(out)
    out = self.dropout(out)

    out = self.fc3(out)
    out = self.relu(out)
    out = self.dropout(out)

    out = self.fc4(out)
    out = self.sigmoid(out)

    return out

# Tarining Process
---


In [67]:
# Initiate the netwrok and the components

input_features = 30

perceptron = Perceptron(input_features)
loss_function = torch.nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.SGD(perceptron.parameters(),lr=config.lr, momentum =0.8)

In [68]:
# Start the trainig process

for epoch in range (config.epochs):
  # inside a single epoch

  for (x_train,y_train)in train_loader:

    # network expecteverything in float
    x_train = x_train.float()
    y_train = y_train.float()

    # forward propoergation
    outputs = perceptron(x_train)

    # compare with acctual output and calculate loss
    loss = loss_function(outputs,y_train)

    # clearing the gradience
    optimizer.zero_grad()

    # backpropergation
    loss.backward()

    # update the weights
    optimizer.step()

  outputs = (outputs > 0.5).float()
  accuracy = (outputs == y_train).float().mean()

  # printing results into wandb

  metrics = {
      "Epoch": (epoch+1)/(config.epochs),
      "Loss": (loss),
      "Accuarcy": accuracy
  }

  wandb.log(metrics)

  print(f"Epochs :{(epoch+1)}/{(config.epochs)}, Loss: {loss}, Accuracy: {accuracy}")

wandb.finish()

Epochs :1/50, Loss: 0.4782509505748749, Accuracy: 0.8333333134651184
Epochs :2/50, Loss: 0.42209112644195557, Accuracy: 0.8518518805503845
Epochs :3/50, Loss: 0.3915393650531769, Accuracy: 0.8148148059844971
Epochs :4/50, Loss: 0.47067567706108093, Accuracy: 0.7037037014961243
Epochs :5/50, Loss: 0.3964124619960785, Accuracy: 0.8148148059844971
Epochs :6/50, Loss: 0.3121173679828644, Accuracy: 0.8888888955116272
Epochs :7/50, Loss: 0.2944342792034149, Accuracy: 0.9074074029922485
Epochs :8/50, Loss: 0.37711188197135925, Accuracy: 0.9259259104728699
Epochs :9/50, Loss: 0.4867304265499115, Accuracy: 0.7592592835426331
Epochs :10/50, Loss: 0.2860307991504669, Accuracy: 0.9074074029922485
Epochs :11/50, Loss: 0.2868085205554962, Accuracy: 0.8888888955116272
Epochs :12/50, Loss: 0.3826362192630768, Accuracy: 0.8148148059844971
Epochs :13/50, Loss: 0.32890284061431885, Accuracy: 0.9074074029922485
Epochs :14/50, Loss: 0.36131393909454346, Accuracy: 0.8518518805503845
Epochs :15/50, Loss: 0.3

Accuarcy,▅▅▄▁▆▇▇▃▆▄▇▅▅▆▃▅▅█▆▅▅▆▆▆▆▇▅▇▇▇█▇██▅▆▅▅▆▇
Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Loss,█▆▆█▄▃▅█▃▆▄▅▅▄▆▅▅▂▃▅▅▃▃▃▄▃▅▂▂▁▁▂▂▁▄▄▅▅▄▂
Accuarcy,0.92593
Epoch,1.0
Loss,0.25069
